# W_GL_BALANCE_FS ETL Process
### Oracle EBS GL_BALANCES to EDW W_GL_BALANCE_FS (Staging with Complex Balance Calculations)

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2623812 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING BASE: Extract GL_BALANCES with period and ledger context
CREATE OR REPLACE TEMP VIEW stg_balances_base AS
SELECT
  bal.ledger_id, bal.code_combination_id, bal.currency_code,
  per.period_name, per.period_set_name, per.period_num, per.start_date, per.end_date,
  per.adjustment_period_flag,
  bal.actual_flag, bal.translated_flag, bal.template_id,
  bal.period_net_dr, bal.period_net_cr,
  bal.begin_balance_dr, bal.begin_balance_cr,
  bal.period_net_dr_beq, bal.period_net_cr_beq,
  bal.begin_balance_dr_beq, bal.begin_balance_cr_beq,
  bal.budget_version_id, bal.encumbrance_type_id,
  bal.last_update_date AS last_update_date_bal,
  per.last_update_date AS last_update_date_periods,
  led.last_update_date AS last_update_date_led,
  led.currency_code AS ledger_currency_code,
  gcc.account_type
FROM workspace.oracle_ebs.GL_BALANCES bal
INNER JOIN workspace.oracle_ebs.GL_PERIODS per
  ON bal.period_name = per.period_name AND bal.period_type = per.period_type
INNER JOIN workspace.oracle_ebs.GL_LEDGERS led
  ON bal.ledger_id = led.ledger_id AND per.period_set_name = led.period_set_name
INNER JOIN workspace.oracle_ebs.GL_CODE_COMBINATIONS gcc
  ON bal.code_combination_id = gcc.code_combination_id
CROSS JOIN etl_params p
WHERE bal.actual_flag IN ('A', 'E')
  AND COALESCE(bal.translated_flag, 'X') IN ('Y', 'X', 'R')
  AND bal.template_id IS NULL
  AND bal.last_update_date >= to_timestamp(p.LAST_EXTRACT_DATE, 'yyyy-MM-dd HH:mm:ss');

In [ ]:
%sql
-- 3) BALANCE CALCULATIONS: Apply debit/credit logic with balance type indicator
CREATE OR REPLACE TEMP VIEW stg_balances_with_type AS
SELECT
  b.*,
  bt.balance_type_indicator,
  -- Period End Balance Calculation
  CASE WHEN bt.balance_type_indicator = 'CREDIT' 
       THEN -(b.begin_balance_cr + b.period_net_cr)
       WHEN bt.balance_type_indicator = 'DEBIT' 
       THEN (b.begin_balance_dr + b.period_net_dr)
  END AS period_end_balance,
  -- Period Net Activity Calculation
  CASE WHEN bt.balance_type_indicator = 'CREDIT' 
       THEN -b.period_net_cr
       WHEN bt.balance_type_indicator = 'DEBIT' 
       THEN b.period_net_dr
  END AS period_net_activity,
  -- Beginning Balance
  CASE WHEN bt.balance_type_indicator = 'CREDIT' 
       THEN -b.begin_balance_cr
       WHEN bt.balance_type_indicator = 'DEBIT' 
       THEN b.begin_balance_dr
  END AS begin_balance,
  -- BEQ (Budget Exchange) Balances
  CASE WHEN bt.balance_type_indicator = 'CREDIT' 
       THEN -(b.begin_balance_cr_beq + b.period_net_cr_beq)
       WHEN bt.balance_type_indicator = 'DEBIT' 
       THEN (b.begin_balance_dr_beq + b.period_net_dr_beq)
  END AS period_end_balance_beq,
  CASE WHEN bt.balance_type_indicator = 'CREDIT' 
       THEN -b.period_net_cr_beq
       WHEN bt.balance_type_indicator = 'DEBIT' 
       THEN b.period_net_dr_beq
  END AS period_net_activity_beq
FROM stg_balances_base b
CROSS JOIN (
  SELECT 'CREDIT' AS balance_type_indicator
  UNION ALL
  SELECT 'DEBIT' AS balance_type_indicator
) bt;

In [ ]:
%sql
-- 4) FINAL TRANSFORMATION: Apply currency balance type logic and create integration ID
CREATE OR REPLACE TEMP VIEW stg_balances_final AS
SELECT
  CAST(b.code_combination_id AS STRING) AS gl_account_id,
  b.end_date AS balance_date,
  b.balance_type_indicator AS db_cr_ind,
  -- Balance in Account Currency
  CASE WHEN b.translated_flag IN ('Y', 'R') OR b.currency_code = 'STAT'
       THEN b.period_end_balance
       ELSE COALESCE(b.period_end_balance_beq, 0)
  END AS balance_acct_amt,
  -- Balance in Local Currency
  CASE WHEN b.translated_flag IS NULL OR b.translated_flag NOT IN ('Y', 'R')
       THEN b.period_end_balance
       ELSE 0
  END AS balance_loc_amt,
  -- Activity in Account Currency
  CASE WHEN b.translated_flag IN ('Y', 'R') OR b.currency_code = 'STAT'
       THEN b.period_net_activity
       ELSE COALESCE(b.period_net_activity_beq, 0)
  END AS activity_acct_amt,
  -- Activity in Local Currency
  CASE WHEN b.translated_flag IS NULL OR b.translated_flag NOT IN ('Y', 'R')
       THEN b.period_net_activity
       ELSE 0
  END AS activity_loc_amt,
  -- Currency Code
  CASE WHEN b.translated_flag IS NULL OR b.translated_flag NOT IN ('Y', 'R')
       THEN 'NULL' ELSE b.currency_code
  END AS acct_curr_code,
  b.ledger_currency_code AS loc_curr_code,
  -- Integration ID (different for actual vs encumbrance)
  CASE WHEN b.actual_flag = 'A' THEN
    CONCAT('GL~', CAST(b.ledger_id AS STRING), '~', CAST(b.code_combination_id AS STRING), '~',
           b.period_name, '~', b.balance_type_indicator, '~', b.actual_flag, '~',
           CASE WHEN b.translated_flag IN ('Y', 'R') THEN 'Y' ELSE 'N' END, '~',
           b.ledger_currency_code, '~', 
           CASE WHEN b.translated_flag IS NULL OR b.translated_flag NOT IN ('Y', 'R') THEN 'NULL' ELSE b.currency_code END, '~',
           CAST(COALESCE(b.template_id, 0) AS STRING))
  ELSE
    CONCAT('GL~', CAST(b.ledger_id AS STRING), '~', CAST(b.code_combination_id AS STRING), '~',
           CAST(COALESCE(b.encumbrance_type_id, 0) AS STRING), '~', b.period_name, '~',
           b.balance_type_indicator, '~', b.actual_flag, '~',
           CASE WHEN b.translated_flag IN ('Y', 'R') THEN 'Y' ELSE 'N' END, '~',
           b.ledger_currency_code, '~',
           CASE WHEN b.translated_flag IS NULL OR b.translated_flag NOT IN ('Y', 'R') THEN 'NULL' ELSE b.currency_code END, '~',
           CAST(COALESCE(b.template_id, 0) AS STRING))
  END AS integration_id,
  CASE WHEN b.translated_flag IN ('Y', 'R') THEN 'Y' ELSE 'N' END AS translated_flag,
  CASE WHEN b.template_id IS NOT NULL THEN 'Y' ELSE 'N' END AS summary_account_flag,
  b.adjustment_period_flag AS adjustment_flag,
  b.actual_flag AS balance_type_flag,
  b.last_update_date_bal AS last_update_date,
  b.last_update_date_led AS aux1_changed_on_dt,
  b.last_update_date_periods AS aux2_changed_on_dt,
  CAST(b.ledger_id AS STRING) AS ledger_id,
  b.period_set_name || '~' || b.period_name AS period_id,
  CASE WHEN b.actual_flag = 'A' THEN 'Y' ELSE 'N' END AS financial_gl_flg,
  'Y' AS budgetary_control_flg,
  -- Balance Type ID
  CASE WHEN b.actual_flag = 'B' THEN 'ACCT_DOC~BUDGETARY CONTROL~B'
       WHEN b.actual_flag = 'A' AND b.account_type = 'E' THEN 'ACCT_DOC~BUDGETARY CONTROL~E'
       WHEN b.actual_flag = 'A' AND b.account_type = 'R' THEN 'ACCT_DOC~BUDGETARY CONTROL~R'
       ELSE 'ACCT_DOC~BUDGETARY CONTROL~X'
  END AS balance_type_id,
  -- Carry Forward Amounts (only for period 1)
  CASE WHEN b.period_num = 1 AND b.translated_flag = 'Y'
       THEN b.begin_balance ELSE 0
  END AS carry_forward_acct_amt,
  CASE WHEN b.period_num = 1 AND b.translated_flag IS NULL
       THEN b.begin_balance ELSE 0
  END AS carry_forward_loc_amt,
  p.DATASOURCE_NUM_ID, p.TENANT_ID, '0' AS x_custom
FROM stg_balances_with_type b
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 5) INSERT: Load data into target table W_GL_BALANCE_FS
INSERT INTO workspace.oracle_edw.w_gl_balance_fs
SELECT 
  gl_account_id, balance_date, db_cr_ind,
  balance_acct_amt, balance_loc_amt, activity_acct_amt, activity_loc_amt,
  acct_curr_code, loc_curr_code, integration_id,
  datasource_num_id, adjustment_flag, balance_type_flag, translated_flag, summary_account_flag,
  last_update_date, aux1_changed_on_dt, aux2_changed_on_dt,
  ledger_id, period_id, period_id AS budget_period_id, ledger_id AS budget_ledger_id,
  financial_gl_flg, budgetary_control_flg,
  carry_forward_acct_amt, carry_forward_loc_amt, balance_type_id,
  tenant_id, x_custom
FROM stg_balances_final;

In [ ]:
%sql
-- 6) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT' AS package_name,
    'W_GL_BALANCE_FS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 7) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT', 'W_GL_BALANCE_FS', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 8) CLEANUP
DROP VIEW IF EXISTS stg_balances_final;
DROP VIEW IF EXISTS stg_balances_with_type;
DROP VIEW IF EXISTS stg_balances_base;
DROP VIEW IF EXISTS etl_params;